In [1]:
from dataClass import DataClass
import pandas as pd

directory = "stock_market_data/sp500/csv"
tickers_csv = "tickers_list.csv"
tickers_df = pd.read_csv(tickers_csv)
# Initialize with your directory and tickers list
data_loader = DataClass(
    directory=directory,
    tickers_csv=tickers_csv
)

# Load the Adjusted Close dataframe
adj_close_df = data_loader.load_adjusted_close()

In [2]:
print(adj_close_df.shape[1])

171


In [5]:
missing_tickers = set(tickers_df.iloc[:, 0].astype(str).str.upper()) - set(adj_close_df.columns[1:])
print(missing_tickers)
print(len(missing_tickers))

{'CI', 'KLAC', 'WFC', 'L', 'QCOM', 'UNH', 'IPG', 'AES', 'ALL', 'NUE', 'ETR', 'BF.B', 'ORCL', 'CVX', 'PFG', 'GLW', 'PGR', 'CMA', 'SYY', 'GS', 'DVN', 'HIG', 'MAS', 'SPGI', 'NKE', 'PNC', 'SNA', 'EXC', 'CVS', 'DHR', 'CCL', 'WMT', 'NEM', 'COF', 'DUK', 'CSX', 'PPL', 'INTC', 'AFL', 'CMCSA', 'MS', 'TGT', 'CMS', 'ETN', 'LLY', 'NVDA', 'ADBE', 'AIG'}
48
